In [7]:
from scipy import ndimage
from scipy.signal import convolve2d
import numpy as np

def eight_neighbor_average_generic_filter(x):
    footprint = np.ones((3, 3))
    # footprint[1, 1] = 0

    result = generic_filter(
        x.astype('float'), np.nanmean, footprint=footprint,
        mode='constant', cval=np.nan)

    return result

def eight_neighbor_average_convolve2d(x):
    kernel = np.ones((3, 3))
    kernel[0, 0] = 0
    kernel[2, 0] = 0
    kernel[0, 2] = 0
    kernel[2, 2] = 0

    neighbor_sum = convolve2d(
        x, kernel, mode='same',
        boundary='fill', fillvalue=0)

    num_neighbor = convolve2d(
        np.ones(x.shape), kernel, mode='same',
        boundary='fill', fillvalue=0)

    return neighbor_sum / num_neighbor

array = np.array([
    [11,12,13,14],
    [21,22,23,24],
    [31,32,33,34],
    [41,42,43,44]
],dtype=float)

averaged = ndimage.generic_filter(array,
                                  np.nanmean,
                                  size=3,
                                  mode='constant',
                                  cval=np.nan)

In [12]:
a = np.random.rand(12000,16000).astype(np.float32)
# a = eight_neighbor_average_convolve2d(array)
a = eight_neighbor_average_convolve2d(a)

In [13]:
del a